In [106]:
import random

class Container:
    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

class Item:
    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

class Packing:
    def __init__(self, containers):
        self.containers = containers
        self.items = []

    def add_item(self, item):
        for container in self.containers:
            if container.width >= item.width and container.height >= item.height and container.depth >= item.depth:
                self.items.append(item)
                self.containers.remove(container)
                new_containers = self.split_container(container, item)
                self.containers.extend(new_containers)
                return True
        return False

    def split_container(self, container, item):
        new_containers = []
        if container.width - item.width > 0:
            new_containers.append(Container(container.width - item.width, container.height, container.depth))
        if container.height - item.height > 0:
            new_containers.append(Container(container.width, container.height - item.height, container.depth))
        if container.depth - item.depth > 0:
            new_containers.append(Container(container.width, container.height, container.depth - item.depth))
        return new_containers

    def fitness(self):
        total_volume = sum([c.width * c.height * c.depth for c in self.containers])
        packed_volume = sum([i.width * i.height * i.depth for i in self.items])
        return packed_volume / total_volume

def init_population(containers, items, population_size):
    population = []
    for i in range(population_size):
        packing = Packing([Container(c.width, c.height, c.depth) for c in containers])
        random.shuffle(items)
        for item in items:
            packing.add_item(item)
        population.append(packing)
    return population

def evaluate_fitness(population):
    for p in population:
        p.fitness()

def selection(population):
    sorted_population = sorted(population, key=lambda x: x.fitness(), reverse=True)
    return sorted_population[0], sorted_population[1]

def crossover(parent1, parent2):
    container = random.choice(parent1.containers)
    index = parent1.containers.index(container)
    new_containers1 = parent1.containers[:index] + parent2.containers[index:]
    new_containers2 = parent2.containers[:index] + parent1.containers[index:]
    child1 = Packing(new_containers1)
    child2 = Packing(new_containers2)
    return child1, child2

def mutation(packing):
    for i in range(len(packing.items)):
        item = packing.items[i]
        if random.random() < 0.1:
            new_item = Item(random.randint(1, item.width), random.randint(1, item.height), random.randint(1, item.depth))
            packing.items[i] = new_item

def genetic_algorithm(containers, items, generations, population_size):
    population = init_population(containers, items, population_size)
    evaluate_fitness(population)
    for i in range(generations):
        parent1, parent2 = selection(population)
        child1, child2 = crossover(parent1, parent2)
        mutation(child1)
        mutation(child2)
        population.extend([child1, child2])
        evaluate_fitness(population)
    best_packing = max(population, key=lambda x: x.fitness())
    return best_packing

def initLists(count_container, count_items):
    containers = []
    items = []
    c_containers = []
    c_items = []
    for i in range(count_container):
        container = [random.randint(5, 30) for j in range(3)]
        containers.append(container)
    
    for i in range(count_items):
        item = [random.randint(5, 30) for j in range(3)]
        items.append(item)
    
    for i in range(len(containers)):
        for j in range(1):
            c_containers.append(Container(width=containers[i][j], height=containers[i][j+1], 
                                          depth=containers[i][j+2]))

    for i in range(len(items)):
        for j in range(1):
            c_items.append(Item(width=items[i][j], height=items[i][j+1], 
                                      depth=items[i][j+2]))
    return c_containers, c_items

In [108]:
%%time

containers, items = initLists(100, 100)

population = init_population(containers=containers, items=items, population_size=10)
best_packing = genetic_algorithm(containers=containers, items=items, generations=100, population_size=10)

print("Containers:")
for i, container in enumerate(best_packing.containers):
    print(f"Container {i+1}: width={container.width}, height={container.height}, depth={container.depth}")
print("Items:")
for item in best_packing.items:
    print(f"Item: width={item.width}, height={item.height}, depth={item.depth}")
print(f"Fitness: {best_packing.fitness()}")

Containers:
Container 1: width=11, height=6, depth=15
Container 2: width=8, height=9, depth=8
Container 3: width=5, height=29, depth=25
Container 4: width=11, height=14, depth=5
Container 5: width=8, height=9, depth=8
Container 6: width=5, height=14, depth=23
Container 7: width=5, height=28, depth=18
Container 8: width=28, height=20, depth=6
Container 9: width=14, height=16, depth=6
Container 10: width=29, height=5, depth=15
Container 11: width=9, height=30, depth=11
Container 12: width=9, height=20, depth=24
Container 13: width=9, height=6, depth=11
Container 14: width=14, height=16, depth=5
Container 15: width=12, height=19, depth=21
Container 16: width=27, height=5, depth=21
Container 17: width=16, height=6, depth=16
Container 18: width=14, height=19, depth=6
Container 19: width=25, height=10, depth=14
Container 20: width=5, height=22, depth=29
Container 21: width=28, height=19, depth=9
Container 22: width=6, height=12, depth=16
Container 23: width=12, height=9, depth=11
Container 2